In [23]:
#загружаю все библитеки
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [24]:
train = pd.read_csv(r'../input/titanic/train.csv')#загружаю данные
test = pd.read_csv(r'../input/titanic/test.csv')

In [25]:
train

In [43]:
sns.barplot(x='Pclass', y='Survived', data=train)

In [45]:
sns.barplot(x='Sex', y='Survived', data=train)

In [47]:
sns.barplot(x='SibSp', y='Survived', data=train)

In [48]:
sns.barplot(x='Embarked', y='Survived', data=train)

In [26]:
train.isnull().sum()

In [27]:
train["Title"] = train["Name"].str.extract('([A-Za-z]+)\.')#выделяем гоноративы
test["Title"] = test["Name"].str.extract('([A-Za-z]+)\.')

train["Title"].value_counts()#колличество гоноративов

In [28]:
def convert_title(title):
    if title in ["Ms", "Mile", "Miss"]:
        return "Miss"
    elif title in ["Mme", "Mrs"]:
        return "Mrs"
    elif title == "Mr":
        return "Mr"
    elif title == "Master":
        return "Master"
    else:
        return "Other"
        
train["Title"] = train["Title"].map(convert_title)
test["Title"] = test["Title"].map(convert_title)

train["Title"].value_counts()
#уменьшаем колличество гоноративов

In [29]:
train.groupby('Title')['Age'].mean()
#смотрим смотрим на средний возраст в зависимости от гоноратива

In [30]:
train.loc[(train.Age.isnull()) & (train["Title"]=='Master'), 'Age'] = 5
train.loc[(train.Age.isnull()) & (train["Title"]=='Miss'), 'Age'] = 22
train.loc[(train.Age.isnull()) & (train["Title"]=='Mr'), 'Age'] = 32
train.loc[(train.Age.isnull()) & (train["Title"]=='Mrs'), 'Age'] = 36
train.loc[(train.Age.isnull()) & (train["Title"]=='Other'), 'Age'] = 44
#заменяем пустые значения

In [31]:
test.loc[(test.Age.isnull()) & (test["Title"]=='Master'), 'Age'] = 5
test.loc[(test.Age.isnull()) & (test["Title"]=='Miss'), 'Age'] = 22
test.loc[(test.Age.isnull()) & (test["Title"]=='Mr'), 'Age'] = 32
test.loc[(test.Age.isnull()) & (test["Title"]=='Mrs'), 'Age'] = 36
test.loc[(test.Age.isnull()) & (test["Title"]=='Other'), 'Age'] = 44

In [32]:
train

In [33]:
df = train[['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Title']]

In [34]:
test

In [35]:
test_pred = test[['PassengerId', 'Pclass', 'Sex', 'Age', 'Fare', 'Title']]#выбираем данные для прогнозирования

In [36]:
df

In [37]:
test_pred

In [38]:
X_train = df.drop("Survived", axis=1)
y_train = df.Survived
X_train = pd.get_dummies(X_train)
X_test = test_pred.copy()
X_test = pd.get_dummies(X_test)

In [39]:
X_test

In [40]:
X_test['Fare'].fillna(value=X_test['Fare'].median(), inplace=True)

In [41]:
X_test.isnull().sum()
#колличество пустых значений

In [42]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_pred_random_forest = clf.predict(X_test)
acc_random_forest = round(clf.score(X_train, y_train) * 100, 2)
print ("Train Accuracy: " + str(acc_random_forest) + '%')
#проведим обучение и смотрим на точность

In [ ]:
output = pd.DataFrame({'PassengerId': test_pred['PassengerId'], 
                       'Survived': y_pred_random_forest})
output.to_csv('submission1.csv', index=False)
#Score: 0.77990 счет на Каггле